In [1]:
import pandas as pd
import numpy as np

In [2]:
fileName = r"C:\Users\ONS_2\Documents\Chris\Data\Copy of Management_information_-_schools_-_1_Sept_2005_to_31_August_2015.xlsx"
df = pd.read_excel(fileName, sheet_name='2005-2015 Inspections', skiprows=0,header=1)

Last inspection of 05-15 sheet has start date of 16/7/15

In [3]:
fileName = r"C:\Users\ONS_2\Documents\Chris\Data\Ofsted_31_December_2015.xlsx"
df2 = pd.read_excel(fileName, sheet_name='D1 Sept to Dec 2015')
df3 = pd.read_excel(fileName, sheet_name='D2 All schools 31 Dec 2015')

In [4]:
# Check that the URNs aren't duplicated in the files
print(len(df2['URN'].unique())== len(df2['URN']))
print(len(df3['URN'].unique())== len(df3['URN']))

True
True


In [5]:
df2['Phase of education'].value_counts()

Primary      494
Secondary    186
Special       27
Nursery       16
PRU           15
Name: Phase of education, dtype: int64

In [6]:
df3['Phase of education'].value_counts()

Primary      16787
Secondary     3382
Special       1041
Nursery        406
PRU            354
Name: Phase of education, dtype: int64

In [7]:
df2sub = df2[df2['Inspection start date']>'2015-07-16']
df3sub = df3[df3['Inspection start date']>'2015-07-16']

In [8]:
len(set(df3sub['URN'])-set(df2sub['URN']))

0

In [9]:
len(set(df2sub['URN'])-set(df3sub['URN']))

177

In [10]:
df3sub['Inspection type'].value_counts()

Requires Improvement S5 Reinspection Visit 1      284
Maintained Academy and School Short inspection    157
Schools into Special Measures Visit 5              21
S8 No Formal Designation Visit                     21
Serious Weaknesses S5 Reinspection                 20
Special Measures S5 ReInspection                   17
Schools into Special Measures Visit 4              12
Schools into Special Measures Visit 3               9
S5 Inspection                                       6
Section 8 Inspection due to Parental Complaint      3
Academy First Section 5                             3
Schools with Serious Weaknesses Visit 3             2
Schools into Special Measures Visit 2               1
Requires Improvement monitoring Visit 1             1
Requires Improvement S5 Reinspection Visit 2        1
Name: Inspection type, dtype: int64

In [11]:
df2sub['Inspection type'].value_counts()

Maintained Academy and School Short inspection    333
Requires Improvement S5 Reinspection Visit 1      285
S8 No Formal Designation Visit                     21
Schools into Special Measures Visit 5              21
Serious Weaknesses S5 Reinspection                 20
Special Measures S5 ReInspection                   17
Schools into Special Measures Visit 4              12
Schools into Special Measures Visit 3               9
S5 Inspection                                       6
Section 8 Inspection due to Parental Complaint      3
Academy First Section 5                             3
Schools with Serious Weaknesses Visit 3             2
Requires Improvement S5 Reinspection Visit 2        1
Requires Improvement monitoring Visit 1             1
Schools into Special Measures Visit 2               1
Name: Inspection type, dtype: int64

In [12]:
df3subInType = df3sub[df3sub['Inspection type']=='Maintained Academy and School Short inspection']
df2subInType = df2sub[df2sub['Inspection type']=='Maintained Academy and School Short inspection']

In [13]:
len(set(df2subInType['Inspection number'])- set(df3subInType['Inspection number']))

176

In [14]:
# Find the 176 rows with inspection number not in df3

df23diff = df2subInType[~df2subInType['Inspection number'].isin(df3subInType['Inspection number'])]

In [15]:
df23diff.to_csv('df23diff.csv')

In [16]:
# Value counts show that the 'Schools - Short inspection' data are not in the df3 (all schools) file
# These are NOT duplicate inspections as the 'Inspection number' is unique for all rows of original data (df2&df3)
# so ***** USE DF2 - SUBSET TAB ******
print(df23diff['Inspection type grouping'].value_counts(),'\n')
print(df2sub['Inspection type grouping'].value_counts())

Schools - Short inspection    176
Name: Inspection type grouping, dtype: int64 

Schools - S5                             332
Schools - Short inspection               176
Schools - Short inspection Conversion    157
Schools - S8 deemed S5                    70
Name: Inspection type grouping, dtype: int64


In [17]:
df3['Inspection number'].describe()

count         21085
unique        21085
top       ITS406070
freq              1
Name: Inspection number, dtype: object

In [18]:
dfToDec15 = df.append(df2, sort=False)

In [19]:
len(df2sub)

735

In [20]:
print(df.shape, df2.shape, dfToDec15.shape)

(66844, 81) (738, 42) (67582, 110)


In [21]:
# Format: Filename, SheetName, skiprows, header
fileNames = {
    'Aug16' : ['Ofsted_31_August_2016.xlsx', 'D1 Sep 15 to Aug 16', None, 0],
    'Aug17' : ['Ofsted_31_August_2017.xlsx', 'D1 Sep 16 to Aug 17', None, 0],
    'Aug18' : ['Ofsted_31_August_2018.xlsx', '1 Sep 17 to 31 Aug 2018', None, 0],
    'Dec15' : ['Ofsted_31_December_2015.xlsx', 'D1 Sept to Dec 2015', None, 0],
    'Dec16' : ['Ofsted_31_December_2016.xlsx', 'D1 Sep 16 to Dec 16', None, 0],
    'Dec17' : ['Ofsted_31_December_2017.xlsx', 'D1 Sep 17 to Dec 17', None, 0],
    'Dec18' : ['Ofsted_31_December_2018.xlsx', '1 Sep 18 to 31 Dec 2018', None, 0],
    'Mar16' : ['Ofsted_31_March_2016.xlsx', 'D1 Sep 15 to Mar 16', None, 0],
    'Mar17' : ['Ofsted_31_March_2017.xlsx', 'D1 Sep 16 to Mar 17', None, 0],
    'Mar18' : ['Ofsted_31_March_2018.xlsx', '1 Sep 17 to 31 Mar 2018', None, 0]}


In [24]:
def appendDataFrameFromFile(df0, infoList, dropCols = False):
    '''Takes original dataframe df0 and appends a dataframe to the bottom of it
    i.e. it just adds in extra rows at the bottom.
    If dropCols is True, it will delete all of the columns apart from the four
    listed in the 'toKeep' list 
    Returns new dataframe'''
    
    fileName = r"C:\Users\ONS_2\Documents\Chris\Data" + '\\' + infoList[0]
    dfToAdd = pd.read_excel(fileName, 
                       sheet_name= infoList[1], 
                       skiprows= infoList[2],
                       header= infoList[3])
   
    if dropCols == True:
        toKeep = ['URN', 'Inspection start date', 'LAESTAB','Inspection number']
        toDrop1 = list(set(dfToAdd.columns) - set(toKeep))
        dfToAdd.drop(labels=toDrop1, axis=1, inplace=True)
        toDrop2 = list(set(df0.columns) - set(toKeep))
        df0.drop(labels=toDrop2, axis=1, inplace=True)
    dfDict[infoList[0][:-5]] = dfToAdd
    
    print('Appending',infoList[0],'with shape:',dfToAdd.shape, 'to df with shape:',df0.shape)
    newDF = df0.append(dfToAdd, sort=False)
    print('Output df has shape',newDF.shape)
    
    return newDF


    
    
    

In [25]:
bigDF = df.copy()
dfDict = {'0515':bigDF}

for key in fileNames.keys():
    bigDF = appendDataFrameFromFile(bigDF, fileNames[key])

#bigDF.to_csv('bigMash2.csv')
#bigDF = pd.read_csv('bigMash1.csv')

Appending Ofsted_31_August_2016.xlsx with shape: (3359, 42) to df with shape: (66844, 81)
Output df has shape (70203, 111)
Appending Ofsted_31_August_2017.xlsx with shape: (5434, 44) to df with shape: (70203, 111)
Output df has shape (75637, 114)
Appending Ofsted_31_August_2018.xlsx with shape: (6131, 54) to df with shape: (75637, 114)
Output df has shape (81768, 126)
Appending Ofsted_31_December_2015.xlsx with shape: (738, 42) to df with shape: (81768, 126)
Output df has shape (82506, 130)
Appending Ofsted_31_December_2016.xlsx with shape: (1481, 50) to df with shape: (82506, 130)
Output df has shape (83987, 136)
Appending Ofsted_31_December_2017.xlsx with shape: (2035, 44) to df with shape: (83987, 136)
Output df has shape (86022, 136)
Appending Ofsted_31_December_2018.xlsx with shape: (1716, 54) to df with shape: (86022, 136)
Output df has shape (87738, 137)
Appending Ofsted_31_March_2016.xlsx with shape: (1906, 41) to df with shape: (87738, 137)
Output df has shape (89644, 137)
App

In [26]:
len(bigDF['Inspection number'].unique())
# 11,000 duplicate inspections...

85387

In [27]:
#InspNosSoFar = df['Inspection number']
#for key in fileNames.keys():
#    fileName = r"C:\Users\ONS_2\Documents\Chris\Data" + '\\' + fileNames[key][0]
#    dfToAdd = pd.read_excel(fileName, 
#                       sheet_name= fileNames[key][1], 
#                       skiprows= fileNames[key][2],
#                       header= fileNames[key][3])
#    
#    print(key)
#    print(dfToAdd[dfToAdd['Inspection number'].isin(InspNosSoFar)])
#    InspNosSoFar.append(list(dfToAdd[~dfToAdd['Inspection number'].isin(InspNosSoFar)]))

In [28]:
dupExampleDF = bigDF[(bigDF['Inspection number']==10020029) | 
                     (bigDF['Inspection number']==10019296)]
dupExampleDF

,Web link,URN,LAESTAB,School name,Post code,Ofsted region,Region,Local authority,Parliamentary constituency,Type of school,...,Inspection type grouping (final),16-19 study programmes,Previous 16-19 study programmes,Linked school type of education,OB number on Roll,conversion decision,deemed flag,Withdrawn date,Previous withdrawn date,Number of warning notices issued in 2018/19 academic year
404,NaN,106774,3713319,Our Lady of Sorrows Catholic Primary School,NaN,"North East, Yorkshire and the Humber",Yorkshire and the Humber,Doncaster,Doncaster Central,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1206,NaN,124445,8604181,King Edward VI High School,NaN,West Midlands,West Midlands,Staffordshire,Stafford,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,NaN,106774,3713319,Our Lady of Sorrows Catholic Primary School,NaN,"North East, Yorkshire and the Humber",Yorkshire and the Humber,Doncaster,Doncaster Central,NaN,...,NaN,NaN,NaN,NaN,149.0,Yes,NaN,NaN,NaN,NaN
995,NaN,124445,8604181,King Edward VI High School,NaN,West Midlands,West Midlands,Staffordshire,Stafford,NaN,...,NaN,NaN,NaN,NaN,720.0,NaN,NaN,NaN,NaN,NaN
432,NaN,106774,3713319,Our Lady of Sorrows Catholic Primary School,NaN,"North East, Yorkshire and the Humber",Yorkshire and the Humber,Doncaster,Doncaster Central,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221,NaN,124445,8604181,King Edward VI High School,NaN,West Midlands,West Midlands,Staffordshire,Stafford,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
dupExampleDF['URN'].nunique()
dupExampleDF['Inspection start date'].nunique()
dupExampleDF['LAESTAB'].nunique()

2

In [30]:
bigDF[bigDF['Inspection number']==10020029][['URN', 'Inspection start date', 'LAESTAB']].nunique()

URN                      1
Inspection start date    1
LAESTAB                  1
dtype: int64

In [31]:
def func(index):
    #print(index)
    return bigDF['Inspection number']==index

In [32]:
#bigDF['Dups'] = bigDF['Inspection number'].apply(func)

In [33]:
#bigDF['Dups']

In [34]:
dfDict.keys()

dict_keys(['0515', 'Ofsted_31_August_2016', 'Ofsted_31_August_2017', 'Ofsted_31_August_2018', 'Ofsted_31_December_2015', 'Ofsted_31_December_2016', 'Ofsted_31_December_2017', 'Ofsted_31_December_2018', 'Ofsted_31_March_2016', 'Ofsted_31_March_2017', 'Ofsted_31_March_2018'])

In [35]:
# No duplicate inspections in each file
tot = 0
for key in dfDict.keys():
    print (key)
    print(len(dfDict[key]), len(dfDict[key]['Inspection number'].unique()))
    tot += len(dfDict[key])
print(tot)

0515
66844 66844
Ofsted_31_August_2016
3359 3359
Ofsted_31_August_2017
5434 5434
Ofsted_31_August_2018
6131 6131
Ofsted_31_December_2015
738 738
Ofsted_31_December_2016
1481 1481
Ofsted_31_December_2017
2035 2035
Ofsted_31_December_2018
1716 1716
Ofsted_31_March_2016
1906 1906
Ofsted_31_March_2017
3216 3216
Ofsted_31_March_2018
4104 4104
96964


In [36]:
96964-85387


11577

In [37]:
66844+3359+5434+6131 

81768

In [38]:
bigDFchopped = df.copy()
dfDictChopped = {'0515':bigDFchopped}

for key in fileNames.keys():
    bigDFchopped = appendDataFrameFromFile(bigDFchopped, fileNames[key], True)

bigDFchopped.to_csv('bigMashchopped.csv')
#bigDF = pd.read_csv('bigMash1.csv')

Appending Ofsted_31_August_2016.xlsx with shape: (3359, 4) to df with shape: (66844, 4)
Output df has shape (70203, 4)
Appending Ofsted_31_August_2017.xlsx with shape: (5434, 4) to df with shape: (70203, 4)
Output df has shape (75637, 4)
Appending Ofsted_31_August_2018.xlsx with shape: (6131, 4) to df with shape: (75637, 4)
Output df has shape (81768, 4)
Appending Ofsted_31_December_2015.xlsx with shape: (738, 4) to df with shape: (81768, 4)
Output df has shape (82506, 4)
Appending Ofsted_31_December_2016.xlsx with shape: (1481, 4) to df with shape: (82506, 4)
Output df has shape (83987, 4)
Appending Ofsted_31_December_2017.xlsx with shape: (2035, 4) to df with shape: (83987, 4)
Output df has shape (86022, 4)
Appending Ofsted_31_December_2018.xlsx with shape: (1716, 4) to df with shape: (86022, 4)
Output df has shape (87738, 4)
Appending Ofsted_31_March_2016.xlsx with shape: (1906, 4) to df with shape: (87738, 4)
Output df has shape (89644, 4)
Appending Ofsted_31_March_2017.xlsx with s

In [39]:
for col in bigDFchopped.columns:
    print(col)
    print(bigDFchopped.duplicated(col).value_counts())

print('\nCount number of fully duplicated rows')
print(bigDFchopped.duplicated().value_counts())

# So all (bar one) of the 11577 duplicated inspection numbers also share the 
# same URN, LAESTAB and Inspection start date.
# So safe to remove duplicate cols

URN
True     68820
False    28144
dtype: int64
LAESTAB
True     71869
False    25095
dtype: int64
Inspection number
False    85387
True     11577
dtype: int64
Inspection start date
True     95012
False     1952
dtype: int64

Count number of fully duplicated rows
False    85388
True     11576
dtype: int64


In [40]:
bigDF.head()

,Web link,URN,LAESTAB,School name,Post code,Ofsted region,Region,Local authority,Parliamentary constituency,Type of school,...,Inspection type grouping (final),16-19 study programmes,Previous 16-19 study programmes,Linked school type of education,OB number on Roll,conversion decision,deemed flag,Withdrawn date,Previous withdrawn date,Number of warning notices issued in 2018/19 academic year
0,Ofsted School Webpage,108120,3837016,St John's Catholic School for the Deaf (Boston...,LS23 6DF,"North East, Yorkshire and Humber",Yorkshire and the Humber,Leeds,Elmet and Rothwell,Non-Maintained Special School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ofsted School Webpage,138793,9262027,Great Yarmouth Primary Academy,NR30 3DT,East of England,East of England,Norfolk,Great Yarmouth,Academy Sponsor Led,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ofsted School Webpage,134897,8963804,The Arches Community Primary School,CH1 5EZ,North West,North West,Cheshire West and Chester,City of Chester,Community School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ofsted School Webpage,114567,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,South East,South East,Brighton and Hove,Hove,Voluntary Aided School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ofsted School Webpage,125524,9372050,Stratford-upon-Avon Primary School,CV37 6HN,West Midlands,West Midlands,Warwickshire,Stratford-on-Avon,Community School,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
print(len(bigDF), len(bigDF['Inspection number'].unique()))

96964 85387


In [42]:
allDupsDF = bigDF[bigDF.duplicated('Inspection number', keep=False)]

print('\nCount number of fully duplicated rows')
print(bigDF.duplicated().value_counts())


Count number of fully duplicated rows
False    96848
True       116
dtype: int64


In [43]:
#allDupsDF.to_csv('allDupsDF.csv')
allDupsDF['Inspection number'].value_counts().describe()

count    8058.000000
mean        2.436709
std         0.496009
min         2.000000
25%         2.000000
50%         2.000000
75%         3.000000
max         3.000000
Name: Inspection number, dtype: float64

In [44]:
bigDFsorted = bigDF.sort_values(by = ['URN','Total number of pupils'], axis=0)
bigDFnoDups = bigDFsorted.drop_duplicates('Inspection number')
print(bigDFnoDups.shape)

(85387, 137)


In [45]:
bigDFnoDups['Inspection number'].nunique()
bigDFnoDups['Overall effectiveness'].describe()

count    85387.000000
mean         3.023353
std          2.162158
min          1.000000
25%          2.000000
50%          2.000000
75%          3.000000
max          9.000000
Name: Overall effectiveness, dtype: float64

In [46]:
URNchanges = pd.read_excel(r"C:\Users\ONS_2\Documents\Chris\Data\Academies.xlsx", sheet_name='Open', skiprows=9)

In [47]:
URNchanges.head(10)

,URN,DFE Number,Academy Name,Academy Phase,Application date/original free school open date,Open,Parliamentary Constituency,Local Authority,Government Office Region,RSC Region,Predecessor School URN(s),Predecessor School Name(s),Academy Route,Trust Type,Trust Name,Trust ID,Companies House Number,Sponsor**,Sponsor ID
0,133386,3096905,Greig City Academy,Secondary,*,2002-09-01,Hornsey and Wood Green,Haringey,London,East of England and North East London,102160,St David and St Katharine CofE Secondary School,Sponsored,SAT,GREIG CITY ACADEMY,TR00916,4220486,Greig Trust,SP00249
1,133768,8066905,Unity City Academy,Secondary,*,2002-09-01,Middlesbrough,Middlesbrough,North East,North,"111738, 111750","Langbaurgh School, Keldholme School",Sponsored,SAT,UNITY CITY ACADEMY TRUST,TR02344,4357009,Academies Enterprise Trust (AET),SP00003
2,134225,2106906,Harris Academy Peckham,Secondary,*,2003-06-03,Camberwell and Peckham,Southwark,London,South East and South London,100848,Warwick Park School,Sponsored,MAT,HARRIS FEDERATION,TR00968,6228587,Harris Federation,SP00257
3,134226,3046905,Capital City Academy,Secondary,*,2003-09-01,Brent Central,Brent,London,North West London and South Central,101551,Willesden High School,Sponsored,SAT,THE CAPITAL CITY ACADEMY TRUST,TR00388,4268208,Sir Frank Lowe,SP00466
4,134369,3076905,Alec Reed Academy,All Through,*,2003-09-01,Ealing North,Ealing,London,North West London and South Central,"101871, 101929","Northolt Primary School, The Compton High Scho...",Sponsored,SAT,THE ALEC REED ACADEMY,TR00039,4444278,Alec Reed,SP00011
5,134224,3526905,Manchester Academy,Secondary,*,2003-09-01,Manchester Central,Manchester,North West,Lancashire and West Yorkshire,105567,Ducie High School,Sponsored,MAT,UNITED LEARNING TRUST,TR02343,4439859,United Learning Trust,SP00592
6,134223,8066906,The King's Academy,Secondary,*,2003-09-01,Middlesbrough South and East Cleveland,Middlesbrough,North East,North,"111743, 111752","Coulby Newham School, Brackenhoe School",Sponsored,MAT,EMMANUEL SCHOOLS FOUNDATION,TR03704,4464331,Emmanuel Schools Foundation,SP00220
7,134253,8926905,Djanogly City Academy,Secondary,*,2003-09-01,Nottingham East,Nottingham,East Midlands,East Midlands and Humber,"122887, 122946","Forest Comprehensive School, Djanogly City Tec...",Sponsored,MAT,DJANOGLY LEARNING TRUST,TR00672,4544722,Djanogly Learning Trust,SP00198
8,134222,2106905,City of London Academy (Southwark),Secondary,*,2003-09-01,Bermondsey and Old Southwark,Southwark,London,South East and South London,NaN,NaN,Sponsored,MAT,CITY OF LONDON ACADEMIES TRUST,TR00514,4504128,City of London Academies Trust,SP00128
9,133697,3356905,Walsall Academy,Secondary,*,2003-09-01,Walsall North,Walsall,West Midlands,West Midlands,104244,T P Riley Community School,Sponsored,MAT,THOMAS TELFORD MULTI ACADEMY TRUST,TR01870,4798185,Thomas Telford School,SP00576


In [52]:
# Add cols showing URN of predecessor school(s)

toKeep = ['Academy Name','Open','Predecessor School URN(s)','URN']
toDrop = set(URNchanges.columns) - set(toKeep)
bigDFnoDups1 = bigDFnoDups.merge(URNchanges, how='left', on='URN')
bigDFnoDups1.drop(toDrop, axis=1, inplace=True)
bigDFnoDups1.to_csv('bigDFnoDups1.csv')

In [49]:
bigDFnoDups1.shape

(85387, 139)

In [50]:
for col in bigDFnoDups1.columns:
    print(col)

Web link
LAESTAB
School name
Post code
Ofsted region
Region
Local authority
Parliamentary constituency
Type of school
Phase of school
Ofsted phase
Dataview phase
Predecessor URN of schools converting to academy
Schools open/closed status
School opening date
Reason for opening
School closing date
Reason for closure
Inspection number
Inspection type
Academic year
Inspection start date
Inspection end date
First published date
Latest published date
Overall effectiveness
Category
Sixth form provision
Early years provision
Pupils' achievement (aggregated)
Achievement of pupils
How well do pupils achieve?
Behaviour and safety of pupils
Quality of teaching
Leadership and management
Overall effectiveness of residential experience (aggregated)
The effectiveness of the boarding provision (pre Jan 2012)
Overall effectiveness of residential experience (post Jan 2012)
Outcomes for residential pupils (post Jan 2012)
Quality of residential provision and care (post Jan 2012)
Residential pupils safety (

In [51]:

bigDFnoDups1[(bigDFnoDups1['Predecessor School URN(s)'])==(bigDFnoDups1['Predecessor URN'])][
    ['Predecessor School URN(s)', 'Predecessor URN']]

print(bigDFnoDups1['Predecessor School URN(s)'].nunique()) # column added from separate sheet
print(bigDFnoDups1['Predecessor URN'].nunique())

print(bigDFnoDups1[bigDFnoDups1['Predecessor School URN(s)'].isnull()]['Predecessor School URN(s)'].count())
#

4347
1468
0


Now have:

bigDFnoDups1(.csv) has a single row for each inspection (so multiple rows for a single school with multiple inspections). If it is an academy that has changed from a school since 2005, the 'Predecessor School URN(s)' col shows the URN(s) of the school(s) that have formed it. 

Now need to get, for each CURRENT school/academy, the number of each type of inspection grading it has received since 2005. This can be used to work out whether the school is stuck.

1. Get list of current schools/academies
2. Find all inspections for them and make new cols
3. Add inspections for previous incarnations of the same school/academy

Make blank dictionary to record no of times with a 1 outcome, 2 outcome, 3 outcome, 4 outcome for each school.
Each dictionary entry to be {URN:[no 1s, no 2s, no 3s, no 4s]}

Write function and use pd.apply:

For given value of URN:
    Find rows with the same URN
    For each of these rows:
        Determine if it is an inspection that counts towards the 4 needed to be stuck
        If so, increment value in dictionary e.g. dict[URN][inspection score e.g. 2] +=1. [Can this all be done by just working through the rows of the table, regardless of repeats?]

After dictionary is full, do 2nd loop where you look for predecessor school
See if URN appears in predecessor URN col (beware commas & multiple entries)
    If so, look for that value in the dictionary
    Add values from list in dictionary to the list in the dictionary for existing URN
    
Use dictionary lists to label schools as stuck or not..